In [1]:
import visual_behavior.database as db
import pandas as pd

## a function to get motion correction qc records

In [2]:
def get_container_qc_records(search_dict={}):
    '''
    find all session level qc records matching the pattern in search_dict
    passing an empty search dict (or no args) returns all records
    returns pandas dataframe
    '''
    conn = db.Database('visual_behavior_data')
    collection = conn['ophys_qc']['container_qc_records']
    res = list(collection.find(search_dict))
    conn.close()
    return pd.DataFrame(res)

## get all records

In [3]:
motion_correction_records = get_container_qc_records({'qc_attribute':'Motion Correction'})
motion_correction_records

,_id,timestamp,username,container_id,experiment_ids,qc_attribute,motion_present,qc_labels,input_text,entry_time_utc,experiment_id
0,ffba77de37c947b0a92946f40bdb2cf9,2021-01-06 17:43:07,None,846213087,848692970.0,Motion Correction,no_motion,none_of_these_problems_in_xy_shift_plot,,2021-01-07 01:46:00.645692,NaN
1,9612b572885c4db6860130fdb607bb59,2021-01-06 17:58:38,None,846213087,849199228.0,Motion Correction,no_motion,none_of_these_problems_in_xy_shift_plot,,2021-01-07 01:58:53.587461,NaN
2,3b55a8c36408435dbd9a567cdab24a80,2021-01-06 18:25:37,None,846213087,850489605.0,Motion Correction,no_motion,none_of_these_problems_in_xy_shift_plot,,2021-01-07 02:25:51.692272,NaN
3,bd679bdb3cd04e719646e179e8bac4e9,2021-01-06 19:25:08,None,846213087,853962969.0,Motion Correction,no_motion,none_of_these_problems_in_xy_shift_plot,,2021-01-07 03:30:11.827832,NaN
4,f621fde327a94ac2a5480f9a067004ba,2021-01-07 11:42:00,marinag,782536745,NaN,Motion Correction,0,other_problem,,2021-01-07 19:42:25.129093,778644591
...,...,...,...,...,...,...,...,...,...,...,...
569,f9af4386ab96466fb34fd970e5aa752a,2021-01-11 16:03:19,sam,1018028052,NaN,Motion Correction,no_motion,no_problems,,2021-01-12 00:03:27.868427,989610986
570,c66abce6aac84476b023e2d0289cbeaf,2021-01-11 16:03:51,sam,1018028052,NaN,Motion Correction,yes_motion,other_problem,mild jitter,2021-01-12 00:04:02.055155,990400777
571,5a847fc6993045f2aa57561dc283509e,2021-01-11 16:04:11,sam,1018028052,NaN,Motion Correction,no_motion,no_problems,,2021-01-12 00:04:46.541888,990681007
572,a4f8853d498c494db3323fcba743f3c2,2021-01-11 16:05:14,sam,1018028052,NaN,Motion Correction,yes_motion,other_problem,mild jitter,2021-01-12 00:05:24.622711,991852001


## identify records where the user failed to select an experiment ID (my bad - my app was supposed to require it but I screwed up!)

In [4]:
def has_experiment_id(row):
    return isinstance(row['experiment_id'], int)
motion_correction_records['has_oeid'] = motion_correction_records.apply(has_experiment_id, axis=1)

## examine data

In [5]:
motion_correction_records['username'].unique()

array([None, 'marinag', 'alex', 'nataliao', 'sam', 'dougo', 'KateR',
       'farzaneh', 'peterg', 'michael', 'iryna', 'sams', 'peterl',
       'Kate R'], dtype=object)

In [6]:
motion_correction_records['motion_present'].unique()

array(['no_motion', 0, 'missing_movie', 'yes_motion',
       'movie_too_noisy_to_tell', 'movie_too_dim_to_tell'], dtype=object)

## get movies with motion

In [7]:
movies_with_motion = motion_correction_records.query('motion_present == "yes_motion" and has_oeid')
len(movies_with_motion)

153

## get movies with no motion and no problems

In [8]:
movies_without_motion = motion_correction_records.query('motion_present == "no_motion" and qc_labels == "no_problems" and has_oeid')
len(movies_without_motion)

229

## combine all sessions with motion and 25 randomly sampled movies without motion

In [9]:
cols_to_save = ['experiment_id','motion_present','qc_labels','input_text','username']
data_to_save = pd.concat([movies_with_motion, movies_without_motion.sample(25, random_state=0)])
data_to_save[cols_to_save]

,experiment_id,motion_present,qc_labels,input_text,username
21,885067826,yes_motion,other_problem,,alex
27,908381686,yes_motion,none_of_these_problems_in_xy_shift_plot,,peterg
43,908381686,yes_motion,anomalous_bump_in_xy_shift_plot,,peterg
59,825623170,yes_motion,other_problem,"motion visible along the x axis, apparently no...",sam
60,930996067,yes_motion,no_problems,,KateR
...,...,...,...,...,...
410,993341877,no_motion,no_problems,,marinag
269,806455766,no_motion,no_problems,,peterl
524,973927944,no_motion,no_problems,,nataliao
528,977937585,no_motion,no_problems,,nataliao


## save to csv

In [11]:
data_to_save[cols_to_save].to_csv('/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2021.01.21_movies_for_dan_to_mc.csv', index=False)